In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
from datetime import timedelta

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
person_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person-headers.txt', delimiter='\t')

In [ ]:
person_columns = person_headers.columns

In [ ]:
event_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/event-headers.txt', delimiter='\t')
event_columns = event_headers.columns

In [ ]:
touch_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/touch_headers.txt', delimiter='\t')
touch_columns = touch_headers.columns

In [ ]:
person_ranking_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking.tsv',
                                     delimiter='\t')
person_ranking_columns = person_ranking_headers.columns

In [ ]:
person_ranking_item_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking_item.tsv', 
                                          delimiter='\t')
person_ranking_item_columns = person_ranking_item_headers.columns

In [ ]:
person_ranking_summary_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person_ranking_summary.tsv', 
                                          delimiter='\t')
person_ranking_summary_columns = person_ranking_summary_headers.columns

In [ ]:
person_ranking_columns

In [ ]:
person_ranking_summary_columns

In [ ]:
person_ranking_summary_headers.head()

In [ ]:
person_identifier = dd.read_csv('Z:\\person_identifier.tsv', delimiter='\t', 
                                names=['person_id', 'customer_id', 'identifier',
                                       'identifier_type', '?', 'active', 'last_modified',
                                       'db_last_modified'])

In [ ]:
person = dd.read_csv('Z:\\person.tsv', delimiter='\t', 
                                names=person_columns)

In [ ]:
# some columns look like floats at first because they are all NA, though they are strings (utm_...)
# and dma_code looks like an int at first but has NA
# for these we need to specify the type to read correctly
touch = dd.read_csv('Z:\\touch.tsv', delimiter='\t', encoding='utf-8',
                    dtype={'utm_medium':str, 'utm_terms':str, 'utm_content':str, 'utm_name':str, 'url_domain':str,
                          'dma_code':str,'customer_id': 'object',
                                           'tbd': 'object',
                                       'touch_duration.1': 'object'},
                    assume_missing=True,
                    names=touch_columns)

In [ ]:
len(touch)

In [ ]:
with open('Z:\\touch.tsv', 'r') as f:
    for i in range(1000000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

even with encoding set to utf-8, we quickly see issues. For instance, a date falls on the field tbd. That date is 2019-03-04 01:54:30.000000. Could we find the lines around it and see if there's anything weird?

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(200000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(160000):
        l = f.readline()
        if i>150000 and i % 1000 == 0:
            print(l)

In [ ]:
with open('Z:\\touch.tsv', 'r', encoding='utf-8') as f:
    for i in range(10000000):
        l = f.readline()
        n_tabs = l.count('\t')
        if n_tabs != 58:
            print('Line %i has %i tabs:' % (i, n_tabs))
            print(repr(l))

So, I think ^ is _very_ interesting!

There are essentially two problems:

1) Some fields have \n or \t inside quotes. That should be easy to handle!

2) Some instances of the thirtieth field, whatever it is, have \n. That seems to me harder to completely fix, and we may end up with those lines broken in two...

Update after the fact: I just fixed lines by hand - mostly deleted the extra lines after some fields. Did this by printing as above and then jumping to the affected line on 010Editor.

In [ ]:
touch_sample = pd.read_csv('../data/raw/acquia_samples/updated_data/touch.tsv', delimiter='\t',
                           dtype={'utm_medium':str, 'utm_terms':str, 'utm_content':str, 'utm_name':str, 'url_domain':str,
                                  'dma_code':str},
                           parse_dates=['touch_date', 'db_last_modified_date', 'db_last_modified_date.1'],
                           names=touch_columns)

In [ ]:
counts_by_user = touch_sample.person_id.value_counts()

In [ ]:
counts_by_user

So, there are very few 1s, even in the sample data, which is surprising. But actually it's not: many touches get duplicated, somehow. See, for instance:

In [ ]:
touch_sample[touch_sample.person_id == 2066689106]

Three of these four are the same. So one task is to deduplicate these touches. To preserve all the data we should try to do this in a smart way, eg get the maximum duration, ignore NaN in favor of rows with that data, etc. For now I will not try to do this in a smart way -- let's try to keep just one of the rows, whichever it is.

TODO: make this better.

In [ ]:
np.abs(touch_sample.touch_date.iloc[2782]-touch_sample.touch_date.iloc[2769])

In [ ]:
touch_sample_person_ids = touch_sample.person_id.unique()

In [ ]:
to_delete = pd.Series(index=touch_sample.index, dtype=bool)

In [ ]:
fuzz = pd.Timedelta('00:00:01')
for id_p in touch_sample_person_ids:
    times = []
    sample_one_id = touch_sample[touch_sample.person_id == id_p]
    for idx, row in sample_one_id.iterrows():
        this_time = row.touch_date
        for t in times:
            if np.abs(t-this_time) < fuzz:
                to_delete.iloc[idx] = True
                break
        if not to_delete.iloc[idx]:
            times.append(this_time)

In [ ]:
np.sum(to_delete)

In [ ]:
touch_sample_deduped = touch_sample[~to_delete]

In [ ]:
counts_by_user = touch_sample_deduped.person_id.value_counts()

In [ ]:
counts_by_user_hist = counts_by_user.value_counts()

In [ ]:
touch.loc[10000].compute()

In [ ]:
touch.tail()

In [ ]:
touch_person_ids = touch.person_id.unique()
to_delete = pd.Series(index=touch.index, dtype=bool)

Some type issue which makes me thing columns go misaligned somewhere in the file? This is strange. Will investigate later. The code below won't run until that's solved:

In [ ]:
fuzz = pd.Timedelta('00:00:01')
for id_p in touch_person_ids:
    times = []
    sample_one_id = touch[touch.person_id == id_p]
    for idx, row in sample_one_id.iterrows():
        this_time = row.touch_date
        for t in times:
            if np.abs(t-this_time) < fuzz:
                to_delete.iloc[idx] = True
                break
        if not to_delete.iloc[idx]:
            times.append(this_time)